In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

import matplotlib.pyplot as plt
import numpy as np

from utils.preprocessing import (
    load_dataset
)

from utils.gan import (
    generator, discriminator,
)

from utils.cyclegan import (
    CycleGan,
    generator_loss, discriminator_loss,
    calc_cycle_loss, identity_loss
)

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()

# Import the data

In [ ]:
# get the number of files in the monet and photo directories
data_dir = ('data/')
MONET_FILES = tf.io.gfile.glob(str(data_dir + 'monet_tfrec/*.tfrec'))
PHOTO_FILES = tf.io.gfile.glob(str(data_dir + 'photo_tfrec/*.tfrec'))

In [ ]:
# print the number of files in each directory
print('Monet TFRecord Files:', len(MONET_FILES))
print('Photo TFRecord Files:', len(PHOTO_FILES))

In [ ]:
monet = load_dataset(filenames=MONET_FILES, batch_size=1)
photo = load_dataset(filenames=PHOTO_FILES, batch_size=1)

In [ ]:
# plot 5 random images from the monet dataset
plt.figure(figsize=(10, 10))
for i, image in enumerate(monet.take(5)):
    plt.subplot(5, 5, i + 1)
    plt.imshow((image[0] * 0.5 + 0.5).numpy())
    plt.axis('off')
plt.title('Sample of The Monet Dataset')
plt.show()

# plot 5 random images from the photo dataset
plt.figure(figsize=(10, 10))
for i, image in enumerate(photo.take(5)):
    plt.subplot(5, 5, i + 1)
    plt.imshow((image[0] * 0.5 + 0.5).numpy())
    plt.axis('off')
plt.title('Sample of The Photo Dataset')
plt.show()


# Creating and training the model

In [ ]:
with strategy.scope():
    G_monet = generator()
    G_photo = generator()
    
    D_monet = discriminator()
    D_photo = discriminator()

In [ ]:
with strategy.scope():
    G_optimizer_monet = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    G_optimizer_photo = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

    D_optimizer_monet = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    D_optimizer_photo = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

In [ ]:
with strategy.scope():
    cycle_gan_model = CycleGan(
        paint_generator=G_monet,
        photo_generator=G_photo,
        paint_discriminator=D_monet,
        photo_discriminator=D_photo,
    )

    cycle_gan_model.compile(
        paint_gen_optimizer=G_optimizer_monet,
        photo_gen_optimizer=G_optimizer_photo,
        paint_disc_optimizer=D_optimizer_monet,
        photo_disc_optimizer=D_optimizer_photo,
        gen_loss_fn=generator_loss,
        disc_loss_fn=discriminator_loss,
        cycle_loss_fn=calc_cycle_loss,
        identity_loss_fn=identity_loss,
    )

In [ ]:
NUM_EPOCHS = 25
with strategy.scope():
    history = cycle_gan_model.fit(
        tf.data.Dataset.zip((monet, photo)),
        epochs=NUM_EPOCHS,
    )

In [ ]:
monet_gen_loss = []
photo_gen_loss = []
monet_disc_loss = []
photo_disc_loss = []
total_cycle_loss = []

print(history.history.keys())
for epoc in range(NUM_EPOCHS):
    monet_gen_loss.append(np.average(history.history['monet_gen_loss'][epoc].flatten()))    
    photo_gen_loss.append(np.average(history.history['photo_gen_loss'][epoc].flatten()))
    monet_disc_loss.append(np.average(history.history['monet_disc_loss'][epoc].flatten()))
    photo_disc_loss.append(np.average(history.history['photo_disc_loss'][epoc].flatten()))

# The loss

In [ ]:
plt.plot(monet_gen_loss, label='Monet Gen Loss')
plt.plot(photo_gen_loss, label='Photo Gen Loss')
plt.plot(monet_disc_loss, label='Monet Disc Loss')
plt.plot(photo_disc_loss, label='Photo Disc Loss')

plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Generator and Discriminator Loss')
plt.show()

# Saving the model

In [ ]:
SAVE_PATH = 'images/cycleGAN/'
G_monet.save(SAVE_PATH + 'G_monet.h5')
G_photo.save(SAVE_PATH + 'G_photo.h5')
D_monet.save(SAVE_PATH + 'D_monet.h5')
D_photo.save(SAVE_PATH + 'D_photo.h5')

# Generate the output images

In [ ]:
import PIL
! mkdir output
! mkdir output/images

In [ ]:
i = 1
for img in photo:
    prediction = G_monet(img, training=False)[0].numpy()
    prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
    im = PIL.Image.fromarray(prediction)
    im.save("output/images" + str(i) + ".jpg")
    i += 1

In [ ]:
import shutil
shutil.make_archive("output/output_cyclegan", 'zip', "output/images")